In [1]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    groq_api_key="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",
    model="llama-3.1-8b-instant",
    temperature=0,
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Nike Careers








































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Langua

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'APLA, Supply Chain Sustainability, Senior',
  'experience': 'R-54169',
  'skills': 'Corporate Services',
  'description': 'Ciudad de México, Ciudad de Mexico, Mexico'},
 {'role': 'Abteilungsleiter/in (BOH Coach) - 37.5H - NFS Wertheim (m/w/d)',
  'experience': 'R-53478',
  'skills': 'Retail Stores',
  'description': 'Wertheim, Baden-Wurttemberg, Germany'},
 {'role': 'Abteilungsleiter/in (Coach) - 100% - NFS Landquart',
  'experience': 'R-41112',
  'skills': 'Retail Stores',
  'description': 'Remote, Landquart, Graubunden, Switzerland'},
 {'role': 'Accounting/Administrative Assistant',
  'experience': 'R-57501',
  'skills': 'Retail Stores',
  'description': 'Edinburgh, Indiana, United States'},
 {'role': 'Addetto vendite/Athlete - Part Time 20h NFS MILANO',
  'experience': 'R-31058',
  'skills': 'Retail Stores',
  'description': 'Milano, Milano, Italy'},
 {'role': 'Analyst, Supply Chain Trading Company',
  'experience': 'R-54896',
  'skills': 'Supply Chain and Logistics',
  '

In [7]:
type(json_res)

list

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [16]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}]]

In [15]:
job = json_res[0]
job['skills']

'Corporate Services'

In [17]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Supply Chain Sustainability with AtliQ's Expertise

Dear [Recipient's Name],

I came across your job posting for an APLA, Supply Chain Sustainability, Senior role in Ciudad de México, and I'm excited to introduce you to AtliQ, a leading AI & Software Consulting company. Our team has a proven track record of empowering enterprises with tailored solutions that drive scalability, process optimization, cost reduction, and heightened overall efficiency.

AtliQ's expertise in Corporate Services aligns perfectly with your requirements. Our capabilities include:

- **Supply Chain Optimization**: We've helped numerous clients streamline their supply chain operations, reducing costs and improving delivery times. For instance, our work with [Client X](https://example.com/full-stack-js-portfolio) resulted in a 25% reduction in logistics costs and a 30% increase in on-time deliveries.

- **Sustainability Solutions**: Our team has developed innovative solutions that enable businesses